# Fundamentals of Machine Learning (CSCI-UA.473)

## Lab 2: Regularized Regression, Logistic Regression and Classification

In [ ]:
# Let's import some packages we'll need.
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import autograd.numpy as np
from autograd import grad
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
matplotlib.__version__

## Polynomial Regression and Overfitting

In [ ]:
# Initialize parameters:
noiseMagnitude = 10 # how much random noise is there?
numData = 8 # how many measurements (samples) of the signal?
numPoints = 1001 
leftRange = -5 
rightRange = 5
x = np.linspace(leftRange,rightRange,numPoints) # determine the location 
# of evenly spaced points from -5 to 5 to use as an x-base

# Determine the functional relationship between x and y in reality (ground truth):
sig = 2 # user determines whether the signal is quadratic (1) or cubic (2)
if sig == 1:
    y1 = x**2 # quadratic function
elif sig == 2:
    y1 = x**3 # cubic function
    
# Compute signal plus noise:
y = y1 + noiseMagnitude * np.random.normal(0,1,numPoints) # signal + noise

# Plot data:
plt.figure(1)
plt.plot(x,y1,color='blue',linewidth=5)
plt.xlabel('X') 
plt.ylabel('Y')  
plt.title('Ground Truth')

plt.figure(2)
plt.plot(x,y,color='blue',linewidth=1)
plt.xlabel('X') 
plt.ylabel('Y')  
plt.title('Signal plus noise')

#Ground truth with noise in one plot
plt.figure(3)
plt.plot(x,y,color='blue',linewidth=1)
plt.plot(x,y1,color='black',linewidth=5)


In [ ]:
#%% Determine the location of the sampling (measuring) points 

# Randomly draw points to sample:
samplingIndices = np.random.randint(1,numPoints,numData) # random points, from anywhere on the signal

# Plot data as a subsample of the noisy signal:
plt.plot(x,y,color='blue',linewidth=1)
plt.plot(x,y1,color='black',linewidth=5)
plt.plot(x[samplingIndices],y[samplingIndices],'o',markersize=4,color='red')
plt.xlim(-5,5) # keep it on the same x-range as before

# Note: Parabola doesn't fit perfectly because there is noise (measurement error). We are
# overfitting to noise. The more noise, the worse this effect is
# In real life, all measurements are contaminated with noise, so overfitting
# to noise is always a concern.

In [ ]:
#%% (Over)fitting successive polynomials and calculating RMSE at each point

rmse = np.array([]) # capture RMSE for each polynomial degree
fig = plt.figure(figsize=(20,10))
for ii in range(numData): # loop through each sampling point
    ax = fig.add_subplot(2,4,ii+1)
    numDegrees = ii+1 # degree of polynomial to fit to our 8 data points
    p = np.polyfit(x[samplingIndices],y[samplingIndices],numDegrees) # returns a vector of coefficients p that minimizes the squared error
    yHat = np.polyval(p,x) # evaluate the polynomial at specific values
    ax.plot(x,yHat,color='blue',linewidth=1)
    ax.plot(x[samplingIndices],y[samplingIndices],'ro',markersize=3)
    error = np.sqrt(np.mean((y[samplingIndices] - yHat[samplingIndices])**2))
    ax.set_title('Degrees: {}'.format(numDegrees) + ', RMSE = {:.3f}'.format(error))
    rmse = np.append(rmse,error) # keep track of RMSE - we will use this later
    ax.set_xlabel('X')
    ax.set_ylabel('Y (Signal + Noise)')
    
fig.suptitle('Fits for different degrees of polynomials')



In [ ]:
#%% Plotting RMSE of the training set as function of polynomial degree
plt.plot(np.linspace(1,numData,numData),rmse)
plt.xlabel('Degree of polynomial')
plt.ylabel('RMSE')

## Cross-Validation

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#%% Leave-one-out procedure to cross-validate the number of terms in the
# model. Note: We randomly pick one of the test points to use to calculate
# the RMSE with. We use the other data points to fit the model
# This method is called "leave one out" and is very computationally
# expensive, as one has to fit the model n-1 times


# Initialize parameters:
numRepeats = 50 # Number of samples - how often are we doing this?
rmse = np.zeros([numRepeats,numData-1]) # Reinitialize RMSE (100x7)
# For each polynomial degree, 100x we are going to randomly pick one of
# the points from the set of 8 and compute the RMSE
# We are then going to fit the model from the remaining (7) points
# This is why we only go up to the 7th degree polynomial

# Compute RMSE on test set:
for ii in range(numRepeats): # Loop from 0 to 99
    testIndex = np.random.randint(0,numData,1) # Randomize test index - pick randint from 0 to 7
    testSet = samplingIndices[testIndex] # Find the test set (= 1 random data point from our 8)
    trainingSet = np.copy([samplingIndices]) # Make copy of sampling indices
    trainingSet = np.delete(trainingSet,testIndex) # Delete the test subset
    for jj in range(numData-1): # Loop from 0 to 6 - for each poly degree
        numDegrees = jj+1 # degrees are from 1 to 7, so add 1 to jj each time
        p = np.polyfit(x[trainingSet],y[trainingSet],numDegrees) # compute coefficients
        yHat = np.polyval(p,x)  # then evaluate
        # Calculate RMSE with the test set (just the single point we randomly chose above):
        rmse[ii,jj] = np.sqrt(np.mean((y[testSet] - yHat[testSet])**2)) # store this in rmse container

# Plot data:
plt.plot(np.linspace(1,numData-1,7),np.mean(rmse,axis=0))
plt.title('Real RMSE as a function of degree of polynomial')
plt.xlabel('Degree of polynomial')
plt.ylabel('RMSE measured only at points left out from building model')   


In [ ]:
# The solution? Where RMSE is minimal
solution = np.amin(np.mean(rmse,axis=0)) # value
index = np.argmin(np.mean(rmse,axis=0)) # index
print('The RMSE is minimal at polynomial of degree: {}'.format(index+1)) 

# Note - the console will give you warnings that the polyfit is poorly conditioned sometimes. 
# That's another dead giveaway that you are overfitting. Too many parameters, not enough data.


## Regularization

Another solution is to use regularization, where the idea is to penalize large coefficients which lead to noisy curves. There are two popular ways to perform regularization, Lasso and Ridge, which we discuss below.

In [ ]:
# Load data:
x = np.genfromtxt('mRegDataX.csv',delimiter=',') # satM satV hoursS gpa appreciation fearM fearT
y = np.genfromtxt('mRegDataY.csv',delimiter=',') # outcome: class score

x_norm = (x - np.mean(x, axis=0))/np.std(x, axis=0)
y_norm = (y - np.mean(y, axis=0))/np.std(y, axis=0)

# Doing the full model and calculating the yhats:
model = LinearRegression().fit(x_norm,y_norm)
b0, b1 = model.intercept_, model.coef_
y_hat = np.dot(b1,x_norm.transpose()) + b0

# Scatter plot between predicted and actual score of full model:
r = np.corrcoef(y_hat,y_norm)
plt.plot(y_hat,y_norm,'o',markersize=5)
plt.xlabel('Predicted grade score')
plt.ylabel('Actual grade score')
plt.title('R: {:.3f}'.format(r[0,1])) 

# 4. Splitting the dataset for cross-validation:
x1 = np.copy(x_norm[0:100,:])
y1 = np.copy(y_norm[0:100])
model = LinearRegression().fit(x1,y1)
b0_1, b1_1 = model.intercept_, model.coef_

x2 = np.copy(x_norm[100:,:])
y2 = np.copy(y_norm[100:])
model = LinearRegression().fit(x2,y2)
b0_2, b1_2 = model.intercept_, model.coef_

# 5. Cross-validation. Using the betas from one dataset, but
# measuring the error with the other dataset
y_hat1 = np.dot(b1_2,x1.transpose()) + b0_2
y_hat2 = np.dot(b1_1,x2.transpose()) + b0_1
rmse1 = np.sqrt(np.mean((y_hat1 - y1)**2))
rmse2 = np.sqrt(np.mean((y_hat2 - y2)**2))
print(rmse1, rmse2)


In [ ]:
# 2. Doing the full model and calculating the yhats:
model = Ridge(alpha=20).fit(x_norm,y_norm)
b0, b1 = model.intercept_, model.coef_
y_hat = np.dot(b1,x_norm.transpose()) + b0

# 3. Scatter plot between predicted and actual score of full model:
r = np.corrcoef(y_hat,y_norm)
plt.plot(y_hat,y_norm,'o',markersize=5)
plt.xlabel('Predicted grade score')
plt.ylabel('Actual grade score')
plt.title('R: {:.3f}'.format(r[0,1])) 

# 4. Splitting the dataset for cross-validation:
x1 = np.copy(x_norm[0:100,:])
y1 = np.copy(y_norm[0:100])
model = Ridge(alpha=20).fit(x1,y1)
b0_1, b1_1 = model.intercept_, model.coef_

x2 = np.copy(x_norm[100:,:])
y2 = np.copy(y_norm[100:])
model = Ridge(alpha=20).fit(x2,y2)
b0_2, b1_2 = model.intercept_, model.coef_

# 5. Cross-validation. Using the betas from one dataset, but
# measuring the error with the other dataset
y_hat1 = np.dot(b1_2,x1.transpose()) + b0_2
y_hat2 = np.dot(b1_1,x2.transpose()) + b0_1
rmse1 = np.sqrt(np.mean((y_hat1 - y1)**2))
rmse2 = np.sqrt(np.mean((y_hat2 - y2)**2))
print(rmse1, rmse2)

In [ ]:
# 2. Doing the full model and calculating the yhats:
model = Lasso(alpha=0.1).fit(x_norm,y_norm)
b0, b1 = model.intercept_, model.coef_
y_hat = np.dot(b1,x_norm.transpose()) + b0

# 3. Scatter plot between predicted and actual score of full model:
r = np.corrcoef(y_hat,y_norm)
plt.plot(y_hat,y_norm,'o',markersize=5)
plt.xlabel('Predicted grade score')
plt.ylabel('Actual grade score')
plt.title('R: {:.3f}'.format(r[0,1])) 

# 4. Splitting the dataset for cross-validation:
x1 = np.copy(x_norm[0:150,:])
y1 = np.copy(y_norm[0:150])
model = Lasso(alpha=0.1).fit(x1,y1)
b0_1, b1_1 = model.intercept_, model.coef_

x2 = np.copy(x_norm[150:,:])
y2 = np.copy(y_norm[150:])
model = Lasso(alpha=0.1).fit(x2,y2)
b0_2, b1_2 = model.intercept_, model.coef_

# 5. Cross-validation. Using the betas from one dataset, but
# measuring the error with the other dataset
y_hat1 = np.dot(b1_2,x1.transpose()) + b0_2
y_hat2 = np.dot(b1_1,x2.transpose()) + b0_1
rmse1 = np.sqrt(np.mean((y_hat1 - y1)**2))
rmse2 = np.sqrt(np.mean((y_hat2 - y2)**2))
print(rmse1, rmse2)

Lets measure the spread of $\beta_{OLS}$ by fitting different random subsets of the data

In [ ]:
nReps = 1000
betasOLS = np.empty([x.shape[1],nReps])*np.NaN
nSample = 100 # Sub-Sampling points
nPop = x.shape[0]  # The full population


In [ ]:
for ii in range(nReps): 
    # Sample nSample number of indexes from our data randomly
    sampleIndices = np.random.randint(0,nPop-1,nSample) 
    trainSamples = x[sampleIndices,:]
    # House price, our "label" is the fourth feature
    labels = trainSamples[:,4]
    # Use remaining features for training
    features = np.delete(trainSamples, 4, 1)
    # The data is varying in scale and variance, subtracting the mean and dividing by the standard deviation is 
    # a common technique use to "normalize" the data
    features_norm = (features - np.mean(features, axis=0))/np.std(features, axis=0)
    labels_norm = (labels - np.mean(labels, axis=0))/np.std(labels, axis=0)
    # Run OLS and store betas:
    ols = LinearRegression().fit(features_norm,labels_norm)
    betasOLS[:,ii] = np.concatenate((np.array([ols.intercept_]), ols.coef_),axis=0)

In [ ]:
nBins = 31
plt.xlabel(r'$\beta_{OLS}$')
plt.ylabel('Count')
plt.title(r'Distribution of $\beta_{OLS}$')
plt.hist(betasOLS[5,:],nBins)
plt.show()

### Now lets apply our first form of regularized regression - Ridge Regression

In [ ]:
# Lets pick a value for lambda randomly
lambd = 200
betasRidge = np.empty([x.shape[1],nReps])*np.NaN
for ii in range(nReps):
    # We repreat our training process, this time using the Ridge model instead of the vanilla LinearRegression
    sampleIndices = np.random.randint(0,nPop-1,nSample) 
    trainSamples = x[sampleIndices,:]
    labels = trainSamples[:,4]
    features = np.delete(trainSamples, 4, 1)
    features_norm = (features - np.mean(features, axis=0))/np.std(features, axis=0)
    labels_norm = (labels - np.mean(labels, axis=0))/np.std(labels, axis=0)
    # Run Ridge and store betas:
    ridge = Ridge(alpha=lambd).fit(features_norm,labels_norm)
    betasRidge[:,ii] = np.concatenate((np.array([ridge.intercept_]), ridge.coef_),axis=0)

In [ ]:
nBins = 31
plt.xlabel(r'$\beta_{Ridge}$')
plt.ylabel('Count')
plt.title(r'Distribution of $\beta_{Ridge}$')
plt.hist(betasRidge[5,:],nBins);

What do you observe? What happens when you change the value of $\lambda$ ? 

### Lets also look at what happens when we use Lasso

In [ ]:
# Lets pick a value for lambda randomly
lambd = 0.1
betasLasso = np.empty([x.shape[1],nReps])*np.NaN
for ii in range(nReps):
    # We repreat our training process, this time using the Ridge model instead of the vanilla LinearRegression
    sampleIndices = np.random.randint(0,nPop-1,nSample) 
    trainSamples = x[sampleIndices,:]
    labels = trainSamples[:,4]
    features = np.delete(trainSamples, 4, 1)
    features_norm = (features - np.mean(features, axis=0))/np.std(features, axis=0)
    labels_norm = (labels - np.mean(labels, axis=0))/np.std(labels, axis=0)
    # Run Ridge and store betas:
    lasso = Lasso(alpha=lambd).fit(features_norm,labels_norm)
    betasLasso[:,ii] = np.concatenate((np.array([lasso.intercept_]), lasso.coef_),axis=0)

In [ ]:
nBins = 31
plt.xlabel(r'$\beta_{Lasso}$')
plt.ylabel('Count')
plt.title(r'Distribution of $\beta_{Lasso}$')
plt.hist(betasLasso[5,:],nBins);

In [ ]:
x.shape, y.shape

## Lets look at how the spread of $\beta$ values from different regression algorithms compare to each other

In [ ]:
lambd = 100
betasOLS = np.empty([x.shape[1]+1,nReps])*np.NaN
betasRidge = np.empty([x.shape[1]+1,nReps])*np.NaN
for ii in range(nReps):
    sampleIndices = np.random.randint(0,nPop-1,nSample) 
    trainSamples = x[sampleIndices,:]
    labels = y[sampleIndices]
    x_norm = (trainSamples - np.mean(trainSamples, axis=0))/np.std(trainSamples, axis=0)
    y_norm = (labels - np.mean(labels, axis=0))/np.std(labels, axis=0)
    # Run OLS and store betas:
    ols = LinearRegression().fit(x_norm,y_norm)
    betasOLS[:,ii] = np.concatenate((np.array([ols.intercept_]), ols.coef_),axis=0)
    # Run Ridge and store betas:
    ridge = Ridge(alpha=lambd).fit(x_norm,y_norm)
    betasRidge[:,ii] = np.concatenate((np.array([ridge.intercept_]), ridge.coef_),axis=0)

In [ ]:
## Add a scatter plot for values of betas before and after regularization 
plt.scatter(betasOLS, betasRidge)
plt.xlabel(r'$\beta_{OLS}$')
plt.ylabel(r'$\beta_{Ridge}$')
plt.title(r'Spread of $\beta_{OLS}$ and $\beta_{Ridge}$')

In [ ]:
lambd = 0.2
betasOLS = np.empty([x.shape[1]+1,nReps])*np.NaN
betasLasso = np.empty([x.shape[1]+1,nReps])*np.NaN
for ii in range(nReps):
    sampleIndices = np.random.randint(0,nPop-1,nSample) 
    trainSamples = x[sampleIndices,:]
    labels = y[sampleIndices]
    x_norm = (trainSamples - np.mean(trainSamples, axis=0))/np.std(trainSamples, axis=0)
    y_norm = (labels - np.mean(labels, axis=0))/np.std(labels, axis=0)
    # Run OLS and store betas:
    ols = LinearRegression().fit(x_norm,y_norm)
    betasOLS[:,ii] = np.concatenate((np.array([ols.intercept_]), ols.coef_),axis=0)
    # Run Ridge and store betas:
    lasso = Lasso(alpha=lambd).fit(x_norm,y_norm)
    betasLasso[:,ii] = np.concatenate((np.array([lasso.intercept_]), lasso.coef_),axis=0)

In [ ]:
## Add a scatter plot for values of betas before and after regularization 
plt.scatter(betasOLS, betasLasso)
plt.xlabel(r'$\beta_{OLS}$')
plt.ylabel(r'$\beta_{Lasso}$')
plt.title(r'Spread of $\beta_{OLS}$ and $\beta_{Lasso}$')

## Now how to pick the right values for $\lambda$?
$\lambda$ in both Ridge and Lasso Regressions is what we call a "hyperparameter". <br/>
Why is $\lambda$ not just a parameter? In other words, why can we not just learn $\lambda$ like we are learning our $\beta$'s ?

In [ ]:
# Using ridge regression to find optimal lambda: a scikit-learn implementation
# Load libraries:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Init parameters:
xTrain, xTest, yTrain, yTest = train_test_split(x_norm, y_norm.reshape(-1,1), test_size=0.2, random_state=0)
lambdas = np.linspace(0.001,200,201)
cont = np.empty([len(lambdas),2])*np.NaN # [lambda error]

for ii in range(len(lambdas)):
    ridgeModel = Ridge(alpha=lambdas[ii]).fit(xTrain, yTrain)
    cont[ii,0] = lambdas[ii]
    error = mean_squared_error(yTest,ridgeModel.predict(xTest),squared=False)
    cont[ii,1] = error

plt.plot(cont[:,0],cont[:,1])
plt.xlabel('Lambda')
plt.ylabel('RMSE')
plt.title('Ridge regression')
plt.show()
print('Optimal lambda:',lambdas[np.argmax(cont[:,1]==np.min(cont[:,1]))])



In [ ]:
#%% 5. Now do the same thing--but with lasso regression
import warnings
warnings.filterwarnings('ignore') # Just to ignore warnings that might be thrown due to artifically formed data.

# Load libraries:
from sklearn.linear_model import Lasso

# Init parameters:
xTrain, xTest, yTrain, yTest = train_test_split(x_norm, y_norm.reshape(-1,1), test_size=0.2, random_state=1)
lambdas = np.logspace(-2,0,201)
cont = np.empty([len(lambdas),2])*np.NaN # [lambda error]

for ii in range(len(lambdas)):
    ridgeModel = Lasso(alpha=lambdas[ii]).fit(xTrain, yTrain)
    cont[ii,0] = lambdas[ii]
    error = mean_squared_error(yTest,ridgeModel.predict(xTest),squared=False)
    cont[ii,1] = error

plt.plot(cont[:,0],cont[:,1])
plt.xlabel('Lambda')
plt.ylabel('RMSE')
plt.title('Lasso regression')
plt.show()
print('Optimal lambda:',lambdas[np.argmax(cont[:,1]==np.min(cont[:,1]))])



### Small Detour : Using Autograd (and more importantly computing gradients!)

Autograd is a package for automatic differentiation.  ``autograd.numpy`` is a wrapper for Numpy which contains the same basic operations.  In most machine learning algorithms we need the gradient of a certain loss function to fit our model to the given data.  Autograd computes this gradient automatically for us so that we may use methods such as gradient descent.  A popular function used in logistic regression as well as neural nets is the sigmoid function

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

The derivative is given by

$$
\sigma'(x) = \frac{e^{-x}}{(1 + e^{-x})^2} = \sigma(x) (1 - \sigma(x))
$$

We could either compute this derivative by hand and then hard code it explicitly, or we could just call autograd.

In [ ]:
# Define the sigmoid function.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Hard code the gradient of the sigmoid function.
def grad_sigmoid(x):
    y = sigmoid(x)
    return y * (1 - y)

# Evaluate the gradient using autograd.
grad_sig = grad(sigmoid)

# Plot the two gradients side by side.
x = np.linspace(-4, 4, 100)


# We'll have 2 plots side by side.
fig = plt.figure(3, figsize = (10, 5))

# First plot the autograd derivative.
ax1 = plt.subplot(121) # create a figure with 1 row and 2 columns and get the axis 
y1 = np.asarray([grad_sig(x[i]) for i in range(len(x))])
ax1.plot(x, y1, 'b')
ax1.set_xlabel(r'$x$')
ax1.set_ylabel(r'$\sigma\'(x)$')
ax1.set_title(r'Autograd $\sigma\'(x)$')

# Second plot the hard-coded derivative.
ax2 = plt.subplot(122)
y2 = grad_sigmoid(x)
ax2.plot(x, y2, 'r')
ax2.set_xlabel(r'$x$')
ax2.set_ylabel(r'$\sigma\'(x)$')
ax2.set_title(r'Hard coded $\sigma\'(x)$')

plt.tight_layout();

## Logistic Regression using Synthetic Data and Autograd

We will play around with the logistic regression model implemented from scratch and trained using autograd on synthetic data. 

### Generate the data and plot it


In [ ]:
# For the plots in the remainder of this notebook we will use a widget to get interactive and better looking plots! 
# follow these instructions to set up matplotlib widget for Jupyter Lab
# https://github.com/matplotlib/jupyter-matplotlib#installation
# Specifically you will need to install the widget using : conda install -c conda-forge ipympl
# To enable the widget run the following two commands :
# jupyter nbextension install --py --symlink --sys-prefix --overwrite ipympl
#jupyter nbextension enable --py --sys-prefix ipympl

%matplotlib widget

import numpy
from tqdm import tqdm
from sklearn.datasets import make_blobs

# random_seed = numpy.random.randint(0,100)
random_seed = 65

x, y = make_blobs(n_samples=2000, centers=2, n_features=2, random_state=random_seed)

fig = plt.figure()
plt.scatter(x[:,0], x[:,1], c=y)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Now we have a set of data points $\{(x_{1}, y_{1}), (x_{2}, y_{2}), ... , (x_{n}, y_{n})\}$, where $x_{i} \in R^{d}$ are the feature vectors and $y_{i} \in \{0, 1\}$ are the class labels.

### Split the data into training set and validation set
We consider the first 1500 points as our training set and the remaining 500 as our validation set. 
<b>Is this correct?</b> 

In [ ]:
x_train, y_train = x[:1500], y[:1500]
x_val, y_val = x[1500:], y[1500:]

# sanity check
assert len(x_train) == len(y_train) == 1500
assert len(x_val) == len(y_val) == 500

# should we check anything else?

### Build the model
Logistic regression model outputs the posterior probability of the class label to be equal to 1. 
$$p_{+} = p(y = 1|x) = \frac{1}{1 + e^{-w \cdot x + b}},$$ 
where $w \in R^{d}$ and $b \in R$.

Sigmoid function is used to convert the output of the linear operation into probabilities. 

In [ ]:
def plot_function(f):
    x = numpy.linspace(-10,10,100)
    y = f(x)
    
    return plt.plot(x,y)

plt.figure()
plt.xlabel('x')
plt.ylabel('y')
plot_function(sigmoid)
plt.grid(True)
plt.show()

Now lets build the logistic regression model. 

In [ ]:
def logistic_unit(x, w, b):
    """This function computes logistic unit as defined $p_{+}$ above
    :param x: input x with n_dim features
    :param w: weght vector
    :param b: bias vector
    """
    
    # operator @ means matrix multiplication in python <3.5
    # https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.matmul.html#numpy.matmul
    
    pre_sigmoid = x @ w + b
    logit = sigmoid(pre_sigmoid)
    
    return logit

### Loss Function
The loss function of the logistic regression is given by: 

$$\mathcal{L}(x,y) = - ( y \cdot \log(p_{+}) + (1 - y) \cdot \log(1 - p_{+}) )$$

Note that $p_{+}$ depends on $x$ and $w$.

In [ ]:
NEAR_ZERO = 1e-12

def loss_function(x, y, w, b):
    """This function computes the loss (distance)
    :param logits: output from logistic unit
    :param labels: target label
    """
    logits = logistic_unit(x,w,b)
    labels = y
    
    loss = -(labels * numpy.log(logits + NEAR_ZERO) + (1 - labels) * numpy.log(1 - logits + NEAR_ZERO))
    
    return loss

Now lets do a sanity check and compute the loss between the target and prediction given a randomly initialized model.

In [ ]:
# sanity check
n_dim=2
w0 = 0.01 * numpy.random.randn(n_dim)
b0 = 0.0

inp = x_train[0]
label = y_train[0]

out = logistic_unit(inp, w0, b0)
assert (out < 1) and (out > 0)  # why?

loss = loss_function(inp, label, w0, b0)
print(loss)

The above loss function is also called the binary cross-entropy loss. As the true label is either 0 or 1, we can rewrite the above equation as two separate equations. <br />
When $y = 1$, the second term in the above equation goes to zero, and the equation reduces to $-\log(p)$. Therefore, when $y = 1$, the binary cross-entropy loss is equal to the negative logarithm of the predicted probability $p$. Similarly, when the true label $y = 0$, the term $y log(p)$ vanishes, and the expression for binary cross-entropy loss reduces to: $-\log(1-p)$. We can plot each of these components below and see how the loss values change for different values of the true label and the predicted probabilities.

In [ ]:
x_arr = np.linspace(0.001,1)
log_x = np.log(x_arr)
fig, axes = plt.subplots(1, 2,figsize=(8,4))
sns.lineplot(ax=axes[0],x=x_arr,y=log_x)
axes[0].set_title('Plot of log(x) in the interval (0,1]')
axes[0].set(xlabel='x', ylabel='log(x)')
sns.lineplot(ax=axes[1],x=x_arr,y=-log_x)
axes[1].set_title('Plot of -log(x) in the interval (0,1]')
axes[1].set(xlabel='x', ylabel='-log(x)')
plt.show()

Putting together the cross entropy loss can be visualized as,

In [ ]:
plt.clf();
fig, axes = plt.subplots(1, 1,figsize=(8,4))

p = np.linspace(0,1,20)
bce_1 = -np.log(p)
bce_0 = -np.log(1-p)
plot1 = sns.lineplot(x=p,y=bce_1,label='True value:1').set(ylim=(0,4))
plot2 = sns.lineplot(x=p,y=bce_0,label='True value:0').set(ylim=(0,4))
plt.xlabel('p')
plt.ylabel('Binary Cross-Entropy Loss')
plt.show()

### Use Autograd for computing the derivatives and train the model
As discussed in the previous lab, with Autograd we do not need to compute the gradients by hand and code it. This is very handy when we have huge DAG (Directed Acyclic Graph) computations.
There is an active line of research in automatic differentiation, curious students are adviced to read this:
http://jmlr.org/papers/volume18/17-468/17-468.pdf

In [ ]:
# Install autograd:
#!conda install -c conda-forge autograd

import autograd.numpy as numpy
import autograd.numpy.random as npr

from autograd import elementwise_grad

import scipy.optimize

In [ ]:
loss_grad_w = elementwise_grad(loss_function, 2) # partial derivative wrt the weights w (3rd input)
loss_grad_b = elementwise_grad(loss_function, 3) # partial derivative wrt the bias b (4th input)

In [ ]:
n_iter = 10000
eta = .0001   # Learning rate
w = numpy.copy(w0)
b = numpy.copy(b0)

loss_log = []

for ni in tqdm(range(n_iter)):
    dw = loss_grad_w(x_train, y_train, w, b)
    db = loss_grad_b(x_train, y_train, w, b)
    w -= eta * dw
    b -= eta * db
    
    loss = numpy.mean(loss_function(x_train, y_train, w, b))
    loss_log.append(loss)

In [ ]:
# lets plot the running element-wise loss value
plt.figure()
plt.plot(loss_log)
plt.xlabel('Iterations')
plt.ylabel('Log Loss')
plt.grid(True)
plt.show()

### Plot the data: the actual data and the model (hyperplane)
We start with first defining some visualization routines and then we do the actual plotting. 

Some visualization routines: 

In [ ]:
# visualize data 
def vis_data(x, y = None, c='r'):
    if y is None: 
        y = [None] * len(x)
    for x_, y_ in zip(x, y):
        if y_ is None:
            plt.plot(x_[0], x_[1], 'o', markerfacecolor='none', markeredgecolor=c)
        else:
            plt.plot(x_[0], x_[1], 'b'+'o' if y_ == 0 else 'y'+'+')
    plt.grid('on')
    
def vis_hyperplane(w, b, typ='k--'):

    lim0 = plt.gca().get_xlim()
    lim1 = plt.gca().get_ylim()
    m0, m1 = lim0[0], lim0[1]

    intercept0 = -(w[0] * m0 + b)/w[1]
    intercept1 = -(w[0] * m1 + b)/w[1]
    
    plt1, = plt.plot([m0, m1], [intercept0, intercept1], typ)

    plt.gca().set_xlim(lim0)
    plt.gca().set_ylim(lim1)
        
    return plt1

def vis_decision_boundary_contour(w, b, typ='k--'):
    
    lim0 = plt.gca().get_xlim()
    lim1 = plt.gca().get_ylim()
    
    x_ = numpy.linspace(lim0[0], lim0[1], 100)
    y_ = numpy.linspace(lim1[0], lim1[1], 100)
    xx, yy = numpy.meshgrid(x_, y_)
    
    x_tra_ = numpy.concatenate([xx.ravel()[:,None], yy.ravel()[:,None]], axis=1)
    
    pred = logistic_unit(x_tra_, w, b)
    plt1 = plt.contourf(xx, yy, pred.reshape(xx.shape), cmap=plot.cm.coolwarm, alpha=0.4)
    
    plt.colorbar(plt1)
    
    plt.gca().set_xlim(lim0)
    plt.gca().set_ylim(lim1)
        
    return plt1

Now plot the data. 

In [ ]:
plt.figure(figsize=(8,8))

vis_data(x, y, c='b')

plt0 = vis_hyperplane(w0, b0, 'k-.')
plt1 = vis_hyperplane(w, b, 'k--')
plt.legend([plt0, plt1], [
        'Initial: ${:.2} x_1 + {:.2} x_2 + {:.2} = 0$'.format(*list(w0)+[b0]),
        'Final: ${:.2} x_1 + {:.2} x_2 + {:.2} = 0$'.format(*list(w)+[b])],
           loc='best')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Logistic Regression for Breast Cancer Classification using Sklearn

We will now implement and train another logistic regression model using Sci-kit learn. The goal will be to implement the model, which given a new data point infers the probability of breast cancer. 

Helper method below copied from: [Helper Method](https://stackoverflow.com/questions/38105539/how-to-convert-a-scikit-learn-dataset-to-a-pandas-dataset)

In [ ]:
# General helper method to convert sci-kit datasets to Pandas DataFrame.
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

Let's eyeball the data a little bit in a quick and hacky manner. Always a good practice to see what the data looks like. **The training data of course!**

In [ ]:
cancer_dataset = datasets.load_breast_cancer() # Load the data and convert to a pandas dataframe
df = sklearn_to_df(cancer_dataset)

print(df.head()) # Print out the first five data points.


Let's gather a few summary statistics about our data. Again, always a good practice. 

In [ ]:
N = len(df) # The number of data points.
print('N = {:d} data points'.format(N))

# Give a barplot of each class.
plt.figure()
plt.bar([0,1], df['target'].value_counts(ascending = True), color = ['r', 'b'], tick_label = cancer_dataset.target_names)
plt.ylabel('Count')
plt.title('Cancer Dataset: Class Counts');

This dataset is unbalanced because there are more examples of benign cancer than malignant.  This is typical of many real-life datasets where we are sometimes limited in how many training examples we have.  Let's split our data into a training and validation set.  We'll use a 80/20 split.

In [ ]:
# Split the data.  DO NOT TOUCH THE TEST DATA FROM HERE ON!!
train_data, val_data = model_selection.train_test_split(df, test_size = 0.2) # 0.2 is 20% validation data.

# Split the features from the class labels.
X_train = train_data.drop('target', axis = 1) # We drop the target from the features.  
X_val  = val_data.drop('target', axis = 1)  # Note that this does not operate inplace.
 
y_train = train_data['target']
y_val  = val_data['target']

Now that our data is loaded and split we can train a logistic regression model.  For the optimization we use the "liblinear" solver.  There are many other solvers that are also available, such as Newton CG for example.  For more information there is a nice stackexchange post here: [Solvers](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [ ]:
# Now fit a logistic regression model.
model = LogisticRegression(solver = 'liblinear')
model.fit(X_train, y_train);

The model is trained so we can validate it on our validation set.  The Sci-kit metrics module contains many useful functions for this purpose.  We try out a few of them below. 

Let us first briefly explain some of these metrics we will use.  

#### Accuracy 
Accuracy is obviously the percentage of all correctly classified examples in our test set.  

#### Confusion Matrix
The confusion matrix is the following matrix:
$$
C = \begin{bmatrix}
\text{Predict 0, Actual 0} & \text{Predict 0, Actual 1}\\
\text{Predict 1, Actual 0} & \text{Predict 1, Actual 1}
\end{bmatrix}
$$
Notice that the diagonal entries are the examples that are correctly classified.  

#### Precision Score
The precision score is the percentage 
$$
\text{Precision } = \frac{C_{00}}{C_{00} + C_{01}}.
$$
So it is the percentage of predicted malignant tumors that we classify correctly.  

#### Recall Score 
The recall score is the percentage
$$
\text{Recall } = \frac{C_{00}}{C_{00} + C_{10}}
$$
So it is the percentage of malignant tumors that we classify correctly. Note that Precision and Recall are two different quantities.

**Using multiple evaluation metrics helps give a better picture of how well our classifier is doing.**

In [ ]:
pred = model.predict(X_val)

# See the percentage of examples that are correctly classified.
accuracy = metrics.accuracy_score(y_val, pred) 
print("Accuracy = {:0.1f}%".format(accuracy * 100))

# The matrix of predictions and true values for each class.
conf_matrix = metrics.confusion_matrix(y_val, pred)
print("Confusion matrix = ")
print(conf_matrix)

# Precision score.
precision = metrics.precision_score(y_val, pred)
print("Precision = {:0.1f}%".format(100 * precision))

# Recall score.
recall = metrics.recall_score(y_val, pred)
print("Recall    = {:0.1f}%".format(100 * recall))

### We can also plot the ROC curve and use it to estimate thresholds

In [ ]:
pred_probs = model.predict_proba(X_val)
metrics.RocCurveDisplay.from_predictions(y_val, pred_probs[:,1])

# or alternatively
metrics.RocCurveDisplay.from_estimator(model, X_val, y_val)

### If we have highly imbalanced classes, we can plot the PR curve as well

In [ ]:
metrics.PrecisionRecallDisplay.from_predictions(y_val, pred_probs[:,1])

# or alternatively
metrics.PrecisionRecallDisplay.from_estimator(model, X_val, y_val)